In [2]:
import os
os.chdir('../')

In [15]:
%pwd

'e:\\Github repositories\\end-to-end-chest-cancer-classification'

In [4]:
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/Aakash-Engineer/end-to-end-chest-cancer-classification.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME'] = 'Aakash-Engineer'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '43180a605413f82a8a1880730357038557dab43f'

In [5]:
import tensorflow as tf

model = tf.keras.models.load_model('artifacts/training/trained_model.h5')

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class EvaluationCinfig:
    model_path: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [16]:
from ChestClassifier.constants import *
from ChestClassifier.utils.common import read_yaml, load_json, save_json, create_directories

class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationCinfig:
        return EvaluationCinfig(
            model_path=self.config.training.trained_model_path,
            training_data='artifacts/data_ingestion/Chest-CT-Scan-data',
            mlflow_uri='https://dagshub.com/Aakash-Engineer/end-to-end-chest-cancer-classification.mlflow',
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )


In [13]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse


class Evaluation:
    def __init__(self, config: EvaluationCinfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            class_mode="sparse",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.model_path)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [17]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-11-16 11:10:44,204: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-11-16 11:10:44,209: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-16 11:10:44,212: INFO: common: created directory at: artifacts]
Found 102 images belonging to 2 classes.
4/4 [==============================] - 10s 614ms/step - loss: 0.2406 - accuracy: 0.5392
[2024-11-16 11:10:55,853: INFO: common: json file saved at: scores.json]


2024/11/16 11:11:00 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-11-16 11:11:03,425: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\palma\AppData\Local\Temp\tmpdj3choqg\model\data\model\assets
[2024-11-16 11:11:07,657: INFO: builder_impl: Assets written to: C:\Users\palma\AppData\Local\Temp\tmpdj3choqg\model\data\model\assets]


e:\Jupyter\envs\eeccc\lib\site-packages\_distutils_hack\__init__.py:31: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Successfully registered model 'VGG16Model'.
2024/11/16 11:15:43 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
